In [1]:
import os
import sys
import gym
import pylab
import random
import numpy as np
from collections import deque
import tensorflow as tf
from tensorflow.keras.layers import Dense
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.initializers import RandomUniform

In [ ]:
class DQN(tf.keras.Model):
    def __init__(self, action_size):
        super(DQN, self).__init__()
        self.fc1 = Dense(24, activation = 'relu')
        self.fc2 = Dense(24, activation = 'relu')
        self.fc_out = Dense(action_size, kernel_initializer = Randomuniform(-1e-3, 1e-3))
        
    def call(self, x):
        x = self.fc1(x)
        x = self.fc2(x)
        q = self.fc_out(x)
        return q

In [2]:
class DQNAgent:
    def __init__(self, state_size, action):
        self.render = False
        
        # 상태와 행동의 크기 정의
        self.state_size = state_size
        self.action_size = action_size
        
        print("39----state_size : ", state_size)
        print("40----action_size : ", action_size)
        
        # DQN 하이퍼 파리미터
        self.discount_factor = 0.99
        self.learning_rate = 0.001
        self.epsilon = 1.0
        self.epsilon_decay = 0.999
        self.epsilon_min = 0.01
        self.batch_size = 64
        self.train_start = 1000
        
        # 리플레이 메모리, 최대 크기
        self.memory = deque(maxlen = 2000)
        
        # 모델과 타깃 모델 생성
        self.model = DQN(action_size)
        self.target_model = DQN(action_size)
        self.optimizer = Adam(lr=self.learning_rate)
        
        # 타깃 모델 초기화
        self.update_target_model()
        
    # 샘플 <s, a, r, s`>을 리플레이 메모리에 저장
    def append_sample(self, state, action, reward, next_state, done):
        self.memory.append((state, action, reward, next_state, done))
        print("78----state, action, reward, next_state, done : ", state, action, reward, next_state, done)
        
    def trina_model(self):
        if self.epsilon > self.epsilon_min:
            self.epsilon *= self.epsilon_decay
            print("84----self.epsilon : ", self.epsilon)
            
        # 메모리에서 배치 크기만큼 무작위로 샘플 추출
        mini_batch = random.sample(self.memory, self.batch_size)
        print("89----mini_batch : ", mini_batch)
        
        states = np.array([sample[0][0] for sample in mini_batch])
        actions = np.array([sample[1] for sample in mini_batch])
        rewards = np.array([sample[2] for sample in mini_batch])
        next_states = np.array([sample[3][0] for sample in mini_batch])
        dones = np.array([sample[4] for sample in mini_batch])
        print("91----states : ", satates)
        print("92----actions : ", actions)
        print("93----rewards : ", rewards)
        print("94----next_states : ", next_states)
        print("95----dones : ", dones)
        
        # 학습 파라메터
        model_params = self.model.trainable_variables
        print("102----model_params : ", model_params)
        with tf.GradientTape() as tape:
            # 현재 상태에 대한 모델의 큐함수
            predicts = self.model(states)
            one_hot_action = tf.one_hot(actions, self.action_size)
            prediects = tf.reduce_sum(one_hot_action * predicts, axis = 1)
            print("106----predicts : ", predicts)
            print("107----one_hot_action : ", one_hot_action)
            print("108----predicts : ", predicts)
            
            # 다음 상태에 대한 타깃 모델의 큐함수
            target_predicts = self.target_model(next_states)
            target_predicts = tf.stop_gradient(target_predicts)
            print("114----target_predicts : ", target_predicts)
            
            # 벨만 최적 방정식을 이용한 어벧이트 타깃
            max_q = np.amax(target_predicts, axis = -1)
            targets = rewards + (1-dones)*self.discount_factor + max_q
            loss = tf.reduce_mean(tf.square(targets - predicts))
            print("120----max_q : ", max_q)
            print("121----targets : ", targets)
            print("122----loss : ", loss)
            
        # 오류함수를 줄이는 방향으로 모델 업데이트
        grads = tape.gradient(loss, model_params)
        self.optimizer.apply_gradients(zip(grads, model_params))
        print("128----grads : ", grads)
        print("129----self.optimizer.apply_gradients : ", self.optimizer.apply_gradients)
        
    def dupate_target_model(self):
        self.target_model.set_weights(self.model.get_weights())
        print("64----self.model.get_weights() : ", self.model.get_weights())

In [ ]:
if __name__ == "__main__":
    # CartPole-v1 환경, 최대 타임스텝 수가 500
    env = gym.make('CartPole-v1')
    state_size = env.observation_space.shape[0]
    action_size = env.action_space.n
    print("137----state_size : ", state_size)
    print("138----action_size : ", )
    
    # DQN 에이전트 생성
    agent = DQNAgent(state_size, action_size)
    
    scores, episodes = [], []
    score_avg = 0
    
    num_episode = 10
    
    for e in range(num_episodesode):
        done = False
        score = 0
        
        # env 초기화
        state = env.reset()
        state = np.reshape(state, [1, state_size])
        print("153----state : ", state)
        
        while not doen:
            if agent.render:
                env.render()
            print("160----state : ", state)
            
            # 현재 상태로 행동을 선택
            action = agent.get_action(state)
            print("161----action : ", action)
            
            # 선택한 행동으로 환경에서 한 타임스텝 진행
            next_state, reward, done, info = env.step(action)
            print("164----next_state, reward, done, info : ", state, reward, done, info)
            next_state = np.reshape(next_state, [1, state_size])
            print("166----next_sate : ", next_sate)
            
            # 타임스텝마다 보상0.1, 에피소드가 중간에 끝나면 -1 보상
            score += reward
            print("170----reward, state : ", reward, state)
            reward = 0.1 if not done or score == 500 else -1
            print("172----reward : ", reward)
            
            # 리플레이 메모리에 샘플 <s, a, r, s`> 저장
            agent.append_sample(state, action, reward, next_state, done)
            
            print("len(agent.memory) : ", len(agent.memory))
            print("agent.train_start : ", agent.train_start)
            
            if len(agent.memory) >= agent.train_start:
                agent.train_model()
            
            if done:
                # 각 에피소드마다 타깃 모델을 모델의 가중치로 업데이트
                agent.update_target_model()
                
                # 에피소드마다 학습 결과 출력
                score_avg = 0.9*score_avg + 0.1*score if score_avg != 0 else score
                print("episode : {:3d} | score avg : {:3.2f} | memory length : {:4d} | epsilon : {:.4f}".format(
                e, score_avg, len(agent.momory), agent.epsilon))